In [1]:
import numpy as np
import argparse
import cv2
import os
import time

In [2]:
labels = open('model/coco.names').read().strip().split('\n')
net = cv2.dnn.readNetFromDarknet('model/yolov3.cfg', 'model/yolov3.weights')
# Create a list of colors for the labels
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
layer_names = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


In [3]:

def extract_boxes_confidences_classids(outputs, confidence, width, height):
    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        for detection in output:            
            # Extract the scores, classid, and the confidence of the prediction
            scores = detection[5:]
            classID = np.argmax(scores)
            conf = scores[classID]
            
            # Consider only the predictions that are above the confidence threshold
            if conf > confidence:
                # Scale the bounding box back to the size of the image
                box = detection[0:4] * np.array([width, height, width, height])
                centerX, centerY, w, h = box.astype('int')

                # Use the center coordinates, width and height to get the coordinates of the top left corner
                x = int(centerX - (w / 2))
                y = int(centerY - (h / 2))

                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(conf))
                classIDs.append(classID)

    return boxes, confidences, classIDs


def draw_bounding_boxes(image, boxes, confidences, classIDs, idxs, colors,SLow):
    if len(idxs) > 0:
        
        
        for i in idxs.flatten():
            # extract bounding box coordinates
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]
            X = isInside(300, 200, 0, 512, 512, 512, x, y) or isInside(300, 200, 0, 512, 512, 512, x + w, y + h) or isInside(300, 200, 0, 512, 512, 512, x+w, y) or isInside(300, 200, 0, 512, 512, 512, x, y + h)
            X_2 = isInside(75, 380, 260, 200, 450, 380, x, y) or isInside(75, 380, 260, 200, 450, 380, x + w, y + h) or isInside(75, 380, 260, 200, 450, 380, x+w, y) or isInside(75, 380, 260, 200, 450, 380, x, y + h)
            p1 = (75, 380)
            p2 = (260, 200)
            p3 = (450, 380)
            cv2.line(image, p1, p2, (255, 0, 0), 3)
            cv2.line(image, p2, p3, (255, 0, 0), 3)
            cv2.line(image, p1, p3, (255, 0, 0), 3)

            # p1 = (300, 200)
            # p2 = (0, 512)
            # p3 = (512, 512)
            # cv2.line(image, p1, p2, (255, 0, 0), 3)
            # cv2.line(image, p2, p3, (255, 0, 0), 3)
            # cv2.line(image, p1, p3, (255, 0, 0), 3)
            area = w*h
            # draw the bounding box and label on the image
            # cv2.rectangle(image, (75, 300), (450, 380), (0,0,255), 2)
            # Y = (x>75 and x<450) and (y>300 and y<380)
            # Y_1 = (x+w>75 and x+w<450) and (y>300 and y<380)
            # Y_2 = (x>75 and x<450) and (y+h>300 and y+h<380)
            # Y_3 = (x+w>75 and x+w<450) and (y+h>300 and y+h<380)
            if(classIDs[i]==0):
                # color = [int(c) for c in colors[classIDs[i]]]
                
                if (X or X_2 ) and area > 2000:
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0,0,255), 2)
                    roi = croped_image[-size-10:-10, -size-10:-10]
                    # Set an index of where the mask is
                    roi[np.where(mask)] = 0
                    roi += Slow
                else:
                    cv2.rectangle(image, (x, y), (x + w, y + h), (255,255,255), 2)

                text = "{}: {:.4f}".format('Person', confidences[i])
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

            if(classIDs[i]==2):
                # color = [int(c) for c in colors[classIDs[i]]]
                if (X or X_2) and area > 2000:
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0,0,255), 2)
                    roi = croped_image[-size-10:-10, -size-10:-10]
                    # Set an index of where the mask is
                    roi[np.where(mask)] = 0
                    roi += Slow
                else:
                    cv2.rectangle(image, (x, y), (x + w, y + h), (255,255,255), 2)

                text = "{}: {:.4f}".format('car', confidences[i])
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

    return image


def make_prediction(net, layer_names, labels, image, confidence, threshold):
    height, width = image.shape[:2]
    
    # Create a blob and pass it through the model
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(layer_names)

    # Extract bounding boxes, confidences and classIDs
    boxes, confidences, classIDs = extract_boxes_confidences_classids(outputs, confidence, width, height)

    # Apply Non-Max Suppression
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence, threshold)

    return boxes, confidences, classIDs, idxs


In [4]:
def area(x1, y1, x2, y2, x3, y3):
 
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1)
                + x3 * (y1 - y2)) / 2.0)

def isInside(x1, y1, x2, y2, x3, y3, x, y):
 
    # Calculate area of triangle ABC
    A = area (x1, y1, x2, y2, x3, y3)
 
    # Calculate area of triangle PBC
    A1 = area (x, y, x2, y2, x3, y3)
     
    # Calculate area of triangle PAC
    A2 = area (x1, y1, x, y, x3, y3)
     
    # Calculate area of triangle PAB
    A3 = area (x1, y1, x2, y2, x, y)
     
    # Check if sum of A1, A2 and A3
    # is same as A
    if(A == A1 + A2 + A3):
        return True
    else:
        return False

In [5]:
cap = cv2.VideoCapture('project.avi')

# Read Slow and resize
Slow = cv2.imread("SlowDown.png")
size = 100
Slow = cv2.resize(Slow, (size, size))
# Create a mask of Slow
img2gray = cv2.cvtColor(Slow, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 1, 255, cv2.THRESH_BINARY)
X = False

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('a.webm',cv2.VideoWriter_fourcc(*'DIVX'),15, frameSize= (512,512))

while cap.isOpened():
    ret, image = cap.read()
    if not ret:
        print('Video file finished.')
        break
    image = cv2.resize(image,(512,512)) 
    croped_image = image#[:,120:400]
    start_time = time.time()
    boxes, confidences, classIDs, idxs = make_prediction(net, layer_names, labels, croped_image, 0.5, 0.3)
    end_time = time.time()
    # for key,value in 
    # print(boxes)
    croped_image = draw_bounding_boxes(croped_image, boxes, confidences, classIDs, idxs, colors,Slow)
    # cv2.putText(img=croped_image, text='FPS: '+str(round(1.0/(end_time-start_time),2)), org=(0, 20), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.5, color=(0, 0, 255),thickness=1)
    
    # Flip the frame
    #croped_image = cv2.flip(croped_image, 1)
    # Region of Image (ROI), where we want to insert logo
    out.write(croped_image)

    cv2.imshow('YOLO Object Detection', croped_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()

cv2.destroyAllWindows()

Video file finished.
